In [11]:
%pip install -U --quiet langchain tiktoken langchain_openai langchain-community

In [12]:
!pip install -Uq langchain-openai

In [18]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo")

def generation(llm, system_message_text):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message_text),
        MessagesPlaceholder(variable_name="messages")
    ])
    chain = prompt | llm | StrOutputParser()
    return chain

system_message = (
    "You are a TOEFL candidate with upper-intermediate English. "
    "Write a response that sounds somewhat human and not too perfect. "
    "Consider the feedback from the TOEFL expert in your revisions."
)
generation_chain = generation(llm, system_message)

def do_reflection(llm):
    reflection_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a TOEFL writing expert. Evaluate the following essay based on TOEFL scoring criteria (0-5) and provide feedback ONLY in the following format:

Score: <number 0-5>
Strengths: <1-2 short strengths>
Weaknesses: <1-2 short weaknesses>
Quick Tips: <1-2 short actionable fixes>
Humanize: <1 suggestion to sound more natural>"""
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    chain = reflection_prompt | llm | StrOutputParser()
    return chain

reflection_chain = do_reflection(llm)

def generate_and_reflect(topic, max_attempts=3):
    messages = [HumanMessage(content=topic)]

    for attempt in range(max_attempts):
        print(f"\n=== Attempt {attempt+1} ===")

        generation_input = {"messages": messages}
        response = generation_chain.invoke(generation_input)
        print("Generated Essay:\n", response, "\n")

        feedback_input = {"messages": [AIMessage(content=response)]}
        feedback = reflection_chain.invoke(feedback_input)
        print("Reflector Feedback:\n", feedback, "\n")


        messages = [
            HumanMessage(content=topic),
            AIMessage(content=response),
            HumanMessage(content=f"Please revise based on this feedback:\n{feedback}")
        ]


In [19]:
if __name__ == "__main__":
    external_topic = "If you could travel anywhere in the world, where would you go and why?"
    generate_and_reflect(external_topic)


=== Attempt 1 ===
Generated Essay:
 I would love to travel to Japan. I've always been fascinated by Japanese culture, the beautiful landscapes, the delicious food, and the mix of tradition and modernity. Visiting places like Tokyo, Kyoto, and Mount Fuji would be a dream come true. Plus, I would love to experience the cherry blossom season in Japan, it looks absolutely stunning! 

Reflector Feedback:
 Score: 4
Strengths: Engaging introduction, clear expression of interest in traveling to Japan
Weaknesses: Lack of specific details or examples related to Japanese culture, food, or landmarks 
Quick Tips: Include specific details or examples to support your points
Humanize: Consider sharing a personal experience or connection to Japan to add depth to your essay 


=== Attempt 2 ===
Generated Essay:
 I've always been drawn to Japan, its captivating culture, diverse landscapes, and delicious cuisine. The blend of ancient traditions with modern innovations fascinates me. My dream is to explor

In [20]:
if __name__ == "__main__":
    external_topic = "What is your favorite book or movie?"
    generate_and_reflect(external_topic)


=== Attempt 1 ===
Generated Essay:
 Hey there! One of my favorite books is "The Great Gatsby" by F. Scott Fitzgerald. I love the way the author captures the glamour and tragedy of the Roaring Twenties. How about you? Do you have a favorite book or movie? 

Reflector Feedback:
 Score: 3
Strengths: The essay is well-organized and the author effectively introduces the topic with a personal anecdote.
Weaknesses: The essay lacks detailed examples or analysis to support the main points.
Quick Tips: Include specific examples or evidence to strengthen your arguments. Try to delve deeper into the analysis of your ideas.
Humanize: Consider sharing more personal experiences or perspectives to connect with the reader on a deeper level. 


=== Attempt 2 ===
Generated Essay:
 Hey! One of the books that I really enjoy is "The Great Gatsby" by F. Scott Fitzgerald. I love how the author portrays the glitz and the dark side of the Jazz Age. It's a fascinating mix of wealth, love, and loss that always k